# Softmax Classifier Eager
- [github](https://github.com/deeplearningzerotoall/TensorFlow/blob/master/tf_2.x/lab-06-1-softmax_classifier-eager.ipynb)
- Tensorflow의 즉시실행(Eager execution)은 그래프 생성 없이 연산을 즉시 실행하는 명령형 프로그래밍 환경이다.
- eager 모드로 진행하면 graph 모드로 진행할 때보다 손쉽게 값을 구할 수 있다.

In [37]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [38]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

tf.random.set_seed(777) # for reproducibility

2.4.1


# Sample Dataset

In [39]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]

y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

# Convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

nb_classes = 3 # number of classes

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


# Softmax function

<img src="https://user-images.githubusercontent.com/64063767/117558150-9593e980-b0b5-11eb-9a93-f0430b779cbb.png" alt="image" style="zoom:50%;" />

In [40]:
# Weight and bias setting
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes, )), name='bias')
variables = [W, b]

print(W, b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


In [41]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[1.3657196e-02 7.9016238e-03 9.7844124e-01]
 [3.9259776e-02 1.7034741e-02 9.4370544e-01]
 [3.8038525e-01 1.6772316e-01 4.5189157e-01]
 [3.2339048e-01 5.9075940e-02 6.1753362e-01]
 [3.6299741e-06 6.2072729e-08 9.9999630e-01]
 [2.6252020e-02 1.0727962e-02 9.6301997e-01]
 [1.5652511e-05 4.2180278e-07 9.9998391e-01]
 [2.9407690e-06 3.8113328e-08 9.9999702e-01]], shape=(8, 3), dtype=float32)


In [42]:
# Softmax one-hot test
sample_db = [[8, 2, 1, 4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[0.9302204  0.06200533 0.00777428]], shape=(1, 3), dtype=float32)


# Cost function

![image](https://user-images.githubusercontent.com/64063767/117558840-7d26cd80-b0bb-11eb-871d-56e2d9a1252b.png)

In [43]:
def cost_function(X, Y):
    logits = hypothesis(X)
    # Cross-Entropy cost function
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    return cost_mean

print(cost_function(x_data, y_data))

tf.Tensor(6.07932, shape=(), dtype=float32)


# Gradient function

In [44]:
# Gradient Test
x = tf.constant(3.0)

with tf.GradientTape() as g:
    g.watch(x)
    y = x ** 2

dy_dx = g.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [45]:
def gradient_function(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_function(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(gradient_function(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.06914607, -0.6509784 ,  0.5818323 ],
       [-1.5221257 , -1.214863  ,  2.7369885 ],
       [-1.2473828 , -1.7611003 ,  3.008483  ],
       [-1.2014606 , -1.8659233 ,  3.0673838 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.15212913, -0.34219202,  0.4943211 ], dtype=float32)>]


# Train & Result

![image](https://user-images.githubusercontent.com/64063767/117566017-3eab0600-b0ef-11eb-9f2a-4b5dd8594e6a.png)


In [46]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

    for epoch in range(epochs):
        grads = gradient_function(X, Y)
        optimizer.apply_gradients(zip(grads, variables))

        if (epoch == 0) or ((epoch+1) % verbose == 0):
            print('Loss at epoch %d: %f' %(epoch+1, cost_function(X, Y).numpy()))

fit(x_data, y_data)

Loss at epoch 1: 2.849417
Loss at epoch 100: 0.684151
Loss at epoch 200: 0.613813
Loss at epoch 300: 0.558205
Loss at epoch 400: 0.508306
Loss at epoch 500: 0.461059
Loss at epoch 600: 0.415072
Loss at epoch 700: 0.369636
Loss at epoch 800: 0.324533
Loss at epoch 900: 0.280720
Loss at epoch 1000: 0.246752
Loss at epoch 1100: 0.232798
Loss at epoch 1200: 0.221645
Loss at epoch 1300: 0.211476
Loss at epoch 1400: 0.202164
Loss at epoch 1500: 0.193606
Loss at epoch 1600: 0.185714
Loss at epoch 1700: 0.178415
Loss at epoch 1800: 0.171645
Loss at epoch 1900: 0.165350
Loss at epoch 2000: 0.159483


# Prediction Check

In [47]:
sample_data = [[2, 1, 3, 2]] # answer_label [[0, 0, 1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1)) # return index 2

tf.Tensor([[0.00112886 0.08154664 0.91732454]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [48]:
score = hypothesis(x_data)

print(score)
print(tf.argmax(score, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[2.1975954e-06 1.2331175e-03 9.9876475e-01]
 [1.1288590e-03 8.1546679e-02 9.1732454e-01]
 [2.2205469e-07 1.6418649e-01 8.3581328e-01]
 [6.3921457e-06 8.5045445e-01 1.4953916e-01]
 [2.6150835e-01 7.2644705e-01 1.2044546e-02]
 [1.3783254e-01 8.6214006e-01 2.7417505e-05]
 [7.4242175e-01 2.5754121e-01 3.6978530e-05]
 [9.2197561e-01 7.8023843e-02 6.0005704e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


# Convert as Calss

In [49]:
class softmax_classifier(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifier, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes, )), name='bias')

    def softmax_function(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)

    def cost_function(self, X, Y):
        logits = self.softmax_function(X)
        cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
        cost_mean = tf.reduce_mean(cost)
        return cost_mean

    def gradient_function(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_function(x_data, y_data)
            grads = tape.gradient(cost, self.variables)
            return grads

    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

        for epoch in range(epochs):
            grads = self.gradient_function(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))

            if (epoch == 0) or ((epoch + 1) % verbose == 0):
                print('Loss at epoch %d: %f' %(epoch+1, self.cost_function(X, Y).numpy()))

model = softmax_classifier(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 2.472669
Loss at epoch 500: 0.375229
Loss at epoch 1000: 0.229923
Loss at epoch 1500: 0.182148
Loss at epoch 2000: 0.150633
